In [1]:
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'
import tensorflow as tf
import numpy as np

In [2]:
embeds = [
    [0.43, 0.15, 0.89], # Your 
    [0.55, 0.87, 0.66], # journey
    [0.57, 0.85, 0.64], # starts
    [0.22, 0.58, 0.33], # with
    [0.77, 0.25, 0.10], # one
    [0.05, 0.80, 0.55]  # step
]

In [3]:
#
#
#

In [4]:
pip install -q torch

Note: you may need to restart the kernel to use updated packages.


In [5]:
# import torch
# import torch.nn as nn
# torch.manual_seed(123)
# d_in = 3
# d_out = 2
# q_weight     = nn.Parameter(torch.rand(d_in, d_out))
# key_weight   = nn.Parameter(torch.rand(d_in, d_out))
# value_weight = nn.Parameter(torch.rand(d_in, d_out))

# print("q_weight:\n", q_weight)

# print("key_weight:\n", key_weight)

# print("value_weight:\n", value_weight)

In [6]:
import torch

inputs = torch.tensor(
  [[0.43, 0.15, 0.89], # Your     (x^1)
   [0.55, 0.87, 0.66], # journey  (x^2)
   [0.57, 0.85, 0.64], # starts   (x^3)
   [0.22, 0.58, 0.33], # with     (x^4)
   [0.77, 0.25, 0.10], # one      (x^5)
   [0.05, 0.80, 0.55]] # step     (x^6)
)
import torch.nn as nn

class SelfAttention_v1(nn.Module):

    def __init__(self, d_in, d_out):
        super().__init__()
        self.W_query = nn.Parameter(torch.rand(d_in, d_out))
        self.W_key   = nn.Parameter(torch.rand(d_in, d_out))
        self.W_value = nn.Parameter(torch.rand(d_in, d_out))

    def forward(self, x):
        keys = x @ self.W_key
        # print("keys:", keys)
        queries = x @ self.W_query
        # print("queries:", queries)
        values = x @ self.W_value
        # print("values:", values) 
        
        attn_scores = queries @ keys.T # omega
        # print("attn_scores:", attn_scores) 
        attn_weights = torch.softmax(
            attn_scores / keys.shape[-1]**0.5, dim=-1
        )
        # print("attn_weights:", attn_weights) 
        context_vec = attn_weights @ values
        return context_vec

d_in = inputs.shape[1] # the input embedding size, d=3
d_out = 2 # the output embedding size, d=2
torch.manual_seed(123)
sa_v1 = SelfAttention_v1(d_in, d_out)
print(sa_v1(inputs))

tensor([[0.2996, 0.8053],
        [0.3061, 0.8210],
        [0.3058, 0.8203],
        [0.2948, 0.7939],
        [0.2927, 0.7891],
        [0.2990, 0.8040]], grad_fn=<MmBackward0>)


In [7]:
# sa_v1.W_query
# sa_v1.W_key
# sa_v1.W_value

In [8]:
inputs = torch.tensor(
  [[0.43, 0.15, 0.89], # Your     (x^1)
   [0.55, 0.87, 0.66], # journey  (x^2)
   [0.57, 0.85, 0.64], # starts   (x^3)
   [0.22, 0.58, 0.33], # with     (x^4)
   [0.77, 0.25, 0.10], # one      (x^5)
   [0.05, 0.80, 0.55]] # step     (x^6)
)
d_in = inputs.shape[1] # the input embedding size, d=3
d_out = 2 # the output embedding size, d=2
batch = torch.stack((inputs, inputs), dim=0)
print(batch.shape) # 2 inputs with 6 tokens each, and each token has embedding dimension 3

torch.Size([2, 6, 3])


In [9]:
class CausalAttention(nn.Module):

    def __init__(self, d_in, d_out, context_length,
                 dropout, qkv_bias=False):
        super().__init__()
        self.d_out = d_out
        self.W_query = nn.Linear(d_in, d_out, bias=qkv_bias)
        self.W_key   = nn.Linear(d_in, d_out, bias=qkv_bias)
        self.W_value = nn.Linear(d_in, d_out, bias=qkv_bias)
        self.dropout = nn.Dropout(dropout) # New
        self.register_buffer('mask', torch.triu(torch.ones(context_length, context_length), diagonal=1)) # New

    def forward(self, x):
        print("x:\n", x)
        b, num_tokens, d_in = x.shape # New batch dimension b
        # For inputs where `num_tokens` exceeds `context_length`, this will result in errors
        # in the mask creation further below.
        # In practice, this is not a problem since the LLM (chapters 4-7) ensures that inputs  
        # do not exceed `context_length` before reaching this forward method. 
        keys = self.W_key(x)        
        queries = self.W_query(x)
        values = self.W_value(x)
        print("values:\n", values)

        attn_scores = queries @ keys.transpose(1, 2) # Changed transpose
        print("attn_scores:\n", attn_scores)
        attn_scores.masked_fill_(  # New, _ ops are in-place
            self.mask.bool()[:num_tokens, :num_tokens], -torch.inf)  # `:num_tokens` to account for cases where the number of tokens in the batch is smaller than the supported context_size
        attn_weights = torch.softmax(
            attn_scores / keys.shape[-1]**0.5, dim=-1
        )
        attn_weights = self.dropout(attn_weights) # New

        context_vec = attn_weights @ values
        return context_vec

torch.manual_seed(123)

context_length = batch.shape[1]
ca = CausalAttention(d_in, d_out, context_length, 0.0)

context_vecs = ca(batch)

print(context_vecs)
print("context_vecs.shape:", context_vecs.shape)

x:
 tensor([[[0.4300, 0.1500, 0.8900],
         [0.5500, 0.8700, 0.6600],
         [0.5700, 0.8500, 0.6400],
         [0.2200, 0.5800, 0.3300],
         [0.7700, 0.2500, 0.1000],
         [0.0500, 0.8000, 0.5500]],

        [[0.4300, 0.1500, 0.8900],
         [0.5500, 0.8700, 0.6600],
         [0.5700, 0.8500, 0.6400],
         [0.2200, 0.5800, 0.3300],
         [0.7700, 0.2500, 0.1000],
         [0.0500, 0.8000, 0.5500]]])
values:
 tensor([[[-0.4519,  0.2216],
         [-0.7142, -0.1961],
         [-0.7127, -0.1971],
         [-0.3809, -0.1557],
         [-0.4861, -0.1597],
         [-0.4213, -0.1501]],

        [[-0.4519,  0.2216],
         [-0.7142, -0.1961],
         [-0.7127, -0.1971],
         [-0.3809, -0.1557],
         [-0.4861, -0.1597],
         [-0.4213, -0.1501]]], grad_fn=<UnsafeViewBackward0>)
attn_scores:
 tensor([[[0.3111, 0.3479, 0.3471, 0.1714, 0.2350, 0.1928],
         [0.1655, 0.2602, 0.2576, 0.1445, 0.1384, 0.1790],
         [0.1667, 0.2602, 0.2577, 0.1443, 0.1391

In [10]:
#
#
#
#

In [11]:
inputs = tf.constant(embeds)
query = inputs[1]
attn_scores_2 = np.empty(inputs.shape[0])

for i, x_i in enumerate(inputs):
    print(i, x_i)
    attn_scores_2[i] = x_i.numpy() @ query.numpy()
attn_scores_2 = tf.constant(attn_scores_2, dtype=tf.float32)    
print("attn_scores_2:", attn_scores_2)

0 tf.Tensor([0.43 0.15 0.89], shape=(3,), dtype=float32)
1 tf.Tensor([0.55 0.87 0.66], shape=(3,), dtype=float32)
2 tf.Tensor([0.57 0.85 0.64], shape=(3,), dtype=float32)
3 tf.Tensor([0.22 0.58 0.33], shape=(3,), dtype=float32)
4 tf.Tensor([0.77 0.25 0.1 ], shape=(3,), dtype=float32)
5 tf.Tensor([0.05 0.8  0.55], shape=(3,), dtype=float32)
attn_scores_2: tf.Tensor([0.95440006 1.4950001  1.4754001  0.8434     0.707      1.0865    ], shape=(6,), dtype=float32)


In [12]:
# attn_weights_2 is softmax of attn_scores_2
attn_weights_2 = tf.nn.softmax(attn_scores_2)
print("attn_weights_2:", attn_weights_2)

attn_weights_2: tf.Tensor([0.13854761 0.23789133 0.23327404 0.1239916  0.10818188 0.15811363], shape=(6,), dtype=float32)


In [13]:
inputs

<tf.Tensor: shape=(6, 3), dtype=float32, numpy=
array([[0.43, 0.15, 0.89],
       [0.55, 0.87, 0.66],
       [0.57, 0.85, 0.64],
       [0.22, 0.58, 0.33],
       [0.77, 0.25, 0.1 ],
       [0.05, 0.8 , 0.55]], dtype=float32)>

In [14]:

context_vec2 = tf.tensordot(attn_weights_2, inputs, axes=1)
context_vec2

<tf.Tensor: shape=(3,), dtype=float32, numpy=array([0.44186577, 0.65148205, 0.56830895], dtype=float32)>

In [15]:
#
#
#
#

In [16]:
class SelfAttenion_v0(tf.keras.layers.Layer):
    def __init__(self):
        super().__init__()
        pass
    
    def __call__(self, x):
        keys = x
        queries = x
        values = x

        attn_scores = tf.matmul(queries, keys, transpose_b=True)
        attn_weights = tf.nn.softmax(attn_scores)
        all_context_vecs = tf.matmul(attn_weights, values, transpose_b=False)
        return all_context_vecs

In [17]:
sa = SelfAttenion_v0()
sa(inputs)

<tf.Tensor: shape=(6, 3), dtype=float32, numpy=
array([[0.44205937, 0.5930985 , 0.578989  ],
       [0.4418658 , 0.65148205, 0.568309  ],
       [0.44312754, 0.6495946 , 0.5670731 ],
       [0.43038973, 0.6298281 , 0.55102706],
       [0.46710175, 0.5909928 , 0.5265966 ],
       [0.41772443, 0.65032315, 0.56453526]], dtype=float32)>

In [18]:
#
#
#


In [19]:
# weights from LLMFS listing 3.1
qw = np.array([[0.2961, 0.5166],
        [0.2517, 0.6886],
        [0.0740, 0.8665]], dtype=np.float32)
kw = np.array([[0.1366, 0.1025],
        [0.1841, 0.7264],
        [0.3153, 0.6871]], dtype=np.float32)
vw = np.array([[0.0756, 0.1966],
        [0.3164, 0.4017],
        [0.1186, 0.8274]], dtype=np.float32)
# dummy_input = tf.zeros(shape=(6, 3))


In [20]:
# Corresponds to LLMFS listing 3.2
class SelfAttenion_v1(tf.keras.layers.Layer):
    def __init__(self):
        super().__init__()
        b = 0
        self.query_layer = tf.keras.layers.Dense(units=2, activation=None, name=f"query-{b}", use_bias=False)
        self.query_layer.build((None, 3 ))
        self.key_layer = tf.keras.layers.Dense(units=2, activation=None, name=f"key-{b}", use_bias=False)
        self.key_layer.build((None, 3))
        self.value_layer = tf.keras.layers.Dense(units=2, activation=None, name=f"value-{b}", use_bias=False)
        self.value_layer.build((None, 3))
    def __call__(self, x):
        keys = self.key_layer(x)
        # print("keys:", keys)
        queries = self.query_layer(x)
        # print("queries:", queries)
        values = self.value_layer(x)
        # print("values:", values)
        attn_scores = tf.matmul(queries, keys, transpose_b=True)
        # print("attn_scores:", attn_scores)
        attn_weights = tf.nn.softmax(attn_scores / keys.shape[-1]**0.5, axis=-1)
        # print("attn_weights:", attn_weights)
        all_context_vecs = tf.matmul(attn_weights, values, transpose_b=False)
        return all_context_vecs

In [21]:
sa = SelfAttenion_v1()
sa.query_layer.set_weights([qw])
sa.key_layer.set_weights([kw])
sa.value_layer.set_weights([vw])
# sa.query_layer.get_config()

# sa.query_layer(dummy_input)
# sa.query_layer.set_weights([qw])
# sa.key_layer.set_weights([kw])
# sa.value_layer.set_weights([vw])

In [22]:
inputs = tf.constant(embeds)
result = sa(inputs)
# LLMFS 3.4.2
np.allclose(result[0], np.array([0.2996, 0.8053]), rtol=1e-3, atol=1e-3)

True

In [23]:
#
#
#
#


In [24]:
import torch
import torch.nn as nn
torch.manual_seed(123)
d_in = 3
d_out = 2
qkv_bias = False
qw = nn.Linear(d_in, d_out, bias=qkv_bias).weight.detach().numpy().T
kw = nn.Linear(d_in, d_out, bias=qkv_bias).weight.detach().numpy().T
vw = nn.Linear(d_in, d_out, bias=qkv_bias).weight.detach().numpy().T

qw = nn.Linear(d_in, d_out, bias=qkv_bias).weight.detach().numpy().T
kw = nn.Linear(d_in, d_out, bias=qkv_bias).weight.detach().numpy().T
vw = nn.Linear(d_in, d_out, bias=qkv_bias).weight.detach().numpy().T

print("qw:\n", qw)
print("kw:\n", kw)
print("vw:\n", vw)

qw:
 [[-0.13615717  0.10756382]
 [ 0.18532233  0.15787685]
 [ 0.4082695   0.55729234]]
kw:
 [[-0.2603904   0.41260317]
 [ 0.18287641  0.4611045 ]
 [-0.25687245 -0.53230095]]
vw:
 [[ 0.49285263  0.23768058]
 [ 0.27569306  0.47995073]
 [ 0.25159022 -0.07623307]]


In [25]:
# corresponds to weights in CausalAttention LLMFS listing 3.3
qw = np.array([[-0.23542964,  0.21772662],
 [ 0.01912448, -0.4919342 ],
 [-0.28674594,  0.42322308]])
kw = np.array([[-0.4196414,   0.2614782 ],
 [-0.45901766, -0.2133264 ],
 [-0.36482018,  0.21605217]])
vw = np.array([[-0.49001414, -0.11346072],
 [-0.35029206, -0.44043937],
 [-0.2119892,   0.37804362]])

In [26]:
qw

array([[-0.23542964,  0.21772662],
       [ 0.01912448, -0.4919342 ],
       [-0.28674594,  0.42322308]])

In [27]:
# import tensorflow as tf

def create_causal_mask(seq_len):
    """
    Generates a boolean causal mask of shape (seq_len, seq_len).
    """
    # Create a tensor of shape (seq_len, seq_len)
    mask = tf.ones((seq_len, seq_len), dtype=tf.bool)
    
    # Use band_part to keep only the lower triangle (including the diagonal)
    # num_lower: -1 means keep all lower-triangular parts
    # num_upper: 0 means keep 0 upper-triangular parts (only the diagonal)
    lower_triangular = tf.linalg.band_part(
        mask, 
        num_lower=-1, 
        num_upper=0
    )
    
    return lower_triangular

In [28]:
# causal_mask = create_causal_mask(6)
# additive_mask = 1.0 - tf.cast(causal_mask, dtype=tf.float32)
# additive_mask
# large_negative_value = -1e9 
# additive_mask_applied = additive_mask * large_negative_value
# additive_mask_applied

In [29]:
# Corresponds to LLMFS listing 3.3
class CausalAttention(tf.keras.layers.Layer):
    def __init__(self):
        super().__init__()
        b = 0
        self.query_layer = tf.keras.layers.Dense(units=2, activation=None, name=f"query-{b}", use_bias=False)
        self.query_layer.build((None, 3))
        self.key_layer = tf.keras.layers.Dense(units=2, activation=None, name=f"key-{b}", use_bias=False)
        self.key_layer.build((None, 3))
        self.value_layer = tf.keras.layers.Dense(units=2, activation=None, name=f"value-{b}", use_bias=False)
        self.value_layer.build((None, 3))
        causal_mask = create_causal_mask(6)
        additive_mask = 1.0 - tf.cast(causal_mask, dtype=tf.float32)
        large_negative_value = -1e9 
        self.additive_mask_applied = additive_mask * large_negative_value
    def __call__(self, x):
        # b, 
        # print("x:\n", x)
        print("x.shape:", x.shape)
        b, num_tokens, d_in = x.shape
        keys = self.key_layer(x)
        # print("self.key_layer:\n", self.key_layer.get_weights())
        # print("keys:\n", keys)
        queries = self.query_layer(x)
        # print("queries:", queries)
        values = self.value_layer(x)
        # print("values:", values)
        # attn_scores = tf.matmul(queries, keys, transpose_b=True)
        coefficients = tf.matmul(queries, tf.transpose(keys, perm=[0,2,1]))
        # coefficients = tf.matmul(queries, tf.transpose(keys, perm=[1,0]))
        # print("coefficients:", coefficients)

        # mask = tf.linalg.band_part(tf.ones((num_tokens, num_tokens)), -1, 0)
        # print("mask:", mask)
        attn_scores = coefficients + self.additive_mask_applied
        # print("attn_scores:", attn_scores)
        attn_weights = tf.nn.softmax(attn_scores / keys.shape[-1]**0.5, axis=-1)
        # print("attn_weights:", attn_weights)
        all_context_vecs = tf.matmul(attn_weights, values, transpose_b=False)
        return all_context_vecs

In [30]:
ca = CausalAttention()
ca.query_layer.set_weights([qw])
ca.key_layer.set_weights([kw])
ca.value_layer.set_weights([vw])

In [31]:
inputs = tf.constant(embeds)
batch = tf.stack((inputs, inputs))
assert batch.shape == [2,6,3]

In [32]:
result = ca(batch)
assert result.shape == [2,6,2]
assert np.allclose(result[0][0], np.array([-0.4519,  0.2216]), rtol=1e-3, atol=1e-3) # LLMFS 3.5.3

x.shape: (2, 6, 3)


In [33]:
class MultiHeadAttentionWrapper(tf.keras.layers.Layer):

    def __init__(self, d_in, d_out, context_length, dropout, num_heads, qkv_bias=False):
        super().__init__()
        self.heads = []
        for i in range(num_heads):
            self.heads.append(CausalAttention())
        

    def __call__(self, x):
        return tf.concat([head(x) for head in self.heads], axis=-1)

In [34]:
#mhaw = MultiHeadAttentionWrapper(d_in=3, d_out=2, context_length=6, dropout=0, num_heads=2, qkv_bias=False)
mhaw = MultiHeadAttentionWrapper(3, 2, 6, 0, num_heads=2, qkv_bias=False)
mhaw.heads[0].query_layer.set_weights([qw])
mhaw.heads[0].key_layer.set_weights([kw])
mhaw.heads[0].value_layer.set_weights([vw])
mhaw.heads[1].query_layer.set_weights([qw])
mhaw.heads[1].key_layer.set_weights([kw])
mhaw.heads[1].value_layer.set_weights([vw])
result = mhaw(batch)

x.shape: (2, 6, 3)
x.shape: (2, 6, 3)


In [35]:
result

<tf.Tensor: shape=(2, 6, 4), dtype=float32, numpy=
array([[[-0.45192027,  0.22160481, -0.45192027,  0.22160481],
        [-0.58743507,  0.0057761 , -0.58743507,  0.0057761 ],
        [-0.63002306, -0.06318259, -0.63002306, -0.06318259],
        [-0.5674567 , -0.08425315, -0.5674567 , -0.08425315],
        [-0.55256176, -0.09806819, -0.55256176, -0.09806819],
        [-0.5299009 , -0.10806764, -0.5299009 , -0.10806764]],

       [[-0.45192027,  0.22160481, -0.45192027,  0.22160481],
        [-0.58743507,  0.0057761 , -0.58743507,  0.0057761 ],
        [-0.63002306, -0.06318259, -0.63002306, -0.06318259],
        [-0.5674567 , -0.08425315, -0.5674567 , -0.08425315],
        [-0.55256176, -0.09806819, -0.55256176, -0.09806819],
        [-0.5299009 , -0.10806764, -0.5299009 , -0.10806764]]],
      dtype=float32)>

In [36]:
#
#
#

In [37]:
# # corresponds to weights in CausalAttention LLMFS listing 3.3
# qw = np.array([[-0.23542964,  0.21772662],
#  [ 0.01912448, -0.4919342 ],
#  [-0.28674594,  0.42322308]])
# kw = np.array([[-0.4196414,   0.2614782 ],
#  [-0.45901766, -0.2133264 ],
#  [-0.36482018,  0.21605217]])
# vw = np.array([[-0.49001414, -0.11346072],
#  [-0.35029206, -0.44043937],
#  [-0.2119892,   0.37804362]])
# out_proj_weight = np.array([[-0.1668, 0.5000], [0.2270,0.1317] ])

# # [[-0.1668,  0.2270],
# #         [ 0.5000,  0.1317]]
# out_proj_bias = np.array([0.1934, 0.6825])

## MultiHeadAttention

Tensorflow implementation of the Pytorch MultiHeadAttention class in section 3.6 of https://github.com/rasbt/LLMs-from-scratch/blob/main/ch03/01_main-chapter-code/ch03.ipynb

In [57]:
class MultiHeadAttention(tf.keras.layers.Layer):
    def __init__(self, d_in, d_out, context_length, dropout, num_heads, qkv_bias=False):
        super().__init__()
        assert (d_out % num_heads == 0), "d_out must be divisible by num_heads"
        self.d_out = d_out
        self.num_heads = num_heads
        self.head_dim = d_out // num_heads
        b = 0
        self.query_layer = tf.keras.layers.Dense(units=d_out, activation=None, name=f"query-{b}", use_bias=qkv_bias)
        self.query_layer.build((None, d_in))
        self.key_layer = tf.keras.layers.Dense(units=d_out, activation=None, name=f"key-{b}", use_bias=qkv_bias)
        self.key_layer.build((None, d_in))
        self.value_layer = tf.keras.layers.Dense(units=d_out, activation=None, name=f"value-{b}", use_bias=qkv_bias)
        self.value_layer.build((None, d_in))

        self.out_proj = tf.keras.layers.Dense(units=d_out, activation=None, name=f"proj-{b}", use_bias=True)
        self.out_proj.build((None, d_out))

        mask = tf.ones((context_length, context_length), dtype=tf.bool) # square matrix of True
        causal_mask = tf.linalg.band_part(mask, num_lower=-1, num_upper=0) # upper right becomes False
        additive_mask = 1.0 - tf.cast(causal_mask, dtype=tf.float32) # upper right becomes 1.0
        self.additive_mask_applied = additive_mask * -1e9   # upper right is large negative value
        
    def __call__(self, x):
        batch_size, num_tokens, d_in = x.shape
        # print(x.shape)
        # print("x:", x.shape, "\n", x)
        assert x.shape == [2,6,3]
        assert np.allclose(x[0][0], np.array([0.43, 0.15, 0.89]), rtol=1e-3, atol=1e-3) # LLMFS 3.5.3

        keys = self.key_layer(x)
        assert keys.shape == [2,6,2]        
        # print("keys:", keys.shape, "\n", keys)

        queries = self.query_layer(x)
        assert queries.shape == [2,6,2]
        # print("queries:", queries.shape, "\n", queries)

        values = self.value_layer(x)
        assert values.shape == [2,6,2]
        # print("values:", values.shape, "\n", values)


        print(".....reshape.......................")
        keys = tf.reshape(keys, [batch_size, num_tokens, self.num_heads, self.head_dim ]) # Unroll last dim: (b, num_tokens, d_out) -> (b, num_tokens, num_heads, head_dim): 2, 6, 2 -> 2, 6, 2, 1
        assert keys.shape == [2,6,2,1]
        # print("keys reshaped:", keys.shape, "\n", keys)
        assert np.allclose(keys[0][0][0], np.array([-0.5740]), rtol=1e-3, atol=1e-3) # LLMFS 3.5.3
        assert np.allclose(keys[0][0][1], np.array([0.2727]), rtol=1e-3, atol=1e-3) # LLMFS 3.5.3



        
        
        queries = tf.reshape(queries, [batch_size, num_tokens, self.num_heads, self.head_dim ]) # 2, 6, 2, 1
        assert queries.shape == [2,6,2,1]
        # print("queries 2:", queries.shape, "\n", queries)
        values = tf.reshape(values, [batch_size, num_tokens, self.num_heads, self.head_dim ]) # 2, 6, 2, 1
        # print("values 2:", values.shape, "\n", values)
        assert values.shape == [2,6,2,1]


        print("..........transpose.............")
        # (b, num_tokens, num_heads, head_dim) -> (b, num_heads, num_tokens, head_dim)
        keys = tf.transpose(keys, perm=[0, 2, 1, 3])      # [2,6,2,1] ->  [2, 2, 6, 1]
        assert keys.shape == [2, 2, 6, 1]
        # print("keys:", keys.shape, "\n", keys) 
        
        values = tf.transpose(values, perm=[0, 2, 1, 3])      # [2,6,2,1] ->  [2, 2, 6, 1]
        assert values.shape == [2, 2, 6, 1]
        # print("values:", values.shape, "\n", values) 

        queries = tf.transpose(queries, perm=[0, 2, 1, 3])      # [2,6,2,1] ->  [2, 2, 6, 1]
        assert queries.shape == [2, 2, 6, 1]
        # print("queries:", queries.shape, "\n", queries) 
        
        # attn_scores = queries @ keys.transpose(2, 3)  # Dot product for each head


        
        print("..........attn_scores.............")
        attn_scores = tf.matmul(queries, tf.transpose(keys, perm=[0, 1, 3, 2]))
        assert attn_scores.shape == [2, 2, 6, 6]
        # print("attn_scores:", attn_scores.shape, "\n", attn_scores)
        assert np.allclose(attn_scores[0][0][0][:3], np.array([ 2.0295e-01,  3.0793e-01,  3.0508e-01]), rtol=1e-3, atol=1e-3) # LLMFS 3.5.3

        print("..........attn_scores after masking.............")
        print("attn_scores.shape:", attn_scores.shape)
        print("self.additive_mask_applied.shape:", self.additive_mask_applied.shape)
        trimmed_additive_mask_applied = self.additive_mask_applied[:num_tokens, :num_tokens]
        print("trimmed_additive_mask_applied.shape:", trimmed_additive_mask_applied.shape)
        attn_scores = attn_scores + trimmed_additive_mask_applied
        # print("attn_scores:", attn_scores)
        attn_weights = tf.nn.softmax(attn_scores / keys.shape[-1]**0.5, axis=-1)
        # print("attn_weights:", attn_weights)

        context_vec = tf.matmul(attn_weights, values)        
        # print("context_vec:", context_vec)
        context_vec = tf.transpose(context_vec, perm=[0, 2, 1, 3])
        # print("context_vec:", context_vec)
        # print(context_vec.shape)
        # print("context_vec:", context_vec)
        context_vec = tf.reshape(context_vec, [batch_size, num_tokens, self.d_out]) # (b, num_tokens, self.d_out)
        print(context_vec.shape)
        assert context_vec.shape == [2, 6, 2]
        context_vec = self.out_proj(context_vec)
        assert context_vec.shape == [2, 6, 2]
        assert np.allclose(context_vec[0][0], np.array([0.3190, 0.4858]), rtol=1e-3, atol=1e-3) # LLMFS 3.5.3
        return context_vec
        

In [58]:
# corresponds to weights in CausalAttention LLMFS listing 3.3
qw = np.array([[-0.23542964,  0.21772662],
 [ 0.01912448, -0.4919342 ],
 [-0.28674594,  0.42322308]])
kw = np.array([[-0.4196414,   0.2614782 ],
 [-0.45901766, -0.2133264 ],
 [-0.36482018,  0.21605217]])
vw = np.array([[-0.49001414, -0.11346072],
 [-0.35029206, -0.44043937],
 [-0.2119892,   0.37804362]])
out_proj_weight = np.array([[-0.1668, 0.5000], [0.2270,0.1317] ])
out_proj_bias = np.array([0.1934, 0.6825])

In [59]:
qw[:2,:2]

array([[-0.23542964,  0.21772662],
       [ 0.01912448, -0.4919342 ]])

In [60]:
inputs = tf.constant(embeds)
batch = tf.stack((inputs, inputs))
assert batch.shape == [2,6,3]
batch_size, context_length, d_in = batch.shape
d_out = 2
mha = MultiHeadAttention(d_in=d_in, d_out=d_out, context_length=context_length+1, dropout=0, num_heads=2)

mha.query_layer.set_weights([qw])
mha.key_layer.set_weights([kw])
mha.value_layer.set_weights([vw])
mha.out_proj.set_weights([out_proj_weight, out_proj_bias])



In [61]:
# kw

In [62]:
context_vecs = mha(batch)

.....reshape.......................
..........transpose.............
..........attn_scores.............
..........attn_scores after masking.............
attn_scores.shape: (2, 2, 6, 6)
self.additive_mask_applied.shape: (7, 7)
trimmed_additive_mask_applied.shape: (6, 6)
(2, 6, 2)


In [63]:
context_vecs

<tf.Tensor: shape=(2, 6, 2), dtype=float32, numpy=
array([[[0.31908458, 0.48572522],
        [0.2944123 , 0.38965017],
        [0.28564087, 0.35925537],
        [0.2693265 , 0.38730368],
        [0.2639324 , 0.39277285],
        [0.25753415, 0.40275958]],

       [[0.31908458, 0.48572522],
        [0.2944123 , 0.38965017],
        [0.28564087, 0.35925537],
        [0.2693265 , 0.38730368],
        [0.2639324 , 0.39277285],
        [0.25753415, 0.40275958]]], dtype=float32)>